In [1]:
from spinbox import *
from itertools import count
seeder = count(0,1) # generates a list of counting numbers for rng "seeds" (not really seeds in Numpy, but indices on random seeds)

# The `spinbox` Tutorial Notebook

First, let me explain the purpose of this code and what it can do. In auxiliary field diffusion Monte Carlo (AFDMC) and Green's function Monte Carlo (GFMC) calculations of atomic nuclei, one approximates the many-body wavefunction with a linear combination of sample "states", which are propagated independently according to the nuclear Hamiltonian. These sample states are tensor product states in AFDMC and linear combinations of tensor product states in GFMC; since the latter is more general, any algorithm emplyed in AFDMC should be also applicable in GFMC in principle. This package provides the foundational elements for these calculations.

At the time of writing, `spinbox` is only concerned with spin-isospin degrees of freedom (i.e. SU(4)). There is an option to tack coordinates on to sample states, but it has not been developed further. One can optionally turn off isospin, so spin up/down is the only variable (i.e. SU(2)).

## GFMC
For GFMC-style calculations one should use `HilbertState` and `HilbertOperator` classes, named as such because the entire basis of the Hilbert space is available. That is, each basis state is a tensor product state.

To start, let's instantiate some random GFMC-style states for a 2-particle system with just spin (no isospin) and compute stuff with them.

In [12]:
state_1 = HilbertState(n_particles=2, ketwise=True, isospin=False).randomize(seed=next(seeder))
print(state_1)

HilbertState ket of 2 particles:
[[ 0.05528649+0.52630379j]
 [-0.15287179+0.33459819j]
 [-0.1207957 -0.09516618j]
 [-0.71397915+0.22629087j]]


We have instatiated a random complex-values vector with 4 components corresponding to the states $|\uparrow\uparrow\rangle, |\uparrow\downarrow\rangle,|\downarrow\uparrow\rangle,|\downarrow\downarrow\rangle$.

Note that `ketwise=True` makes these ket states. Setting this to false makes bra states. Applying the `.dagger()` method conjugates this attribute.

We can instatiate another state and do some usual calculations.

In [13]:
print("< state 1 | state 1 > = " , state_1.dagger() * state_1)

state_2 = HilbertState(n_particles=2, ketwise=False, isospin=False).randomize(seed=next(seeder))

print("< state 2 | state 1 > = " , state_2 * state_1)

< state 1 | state 1 > =  [[1.+0.j]]
< state 2 | state 1 > =  [[0.26070267+0.11864119j]]


The `*` operator does most sensible multiplication. States are normalized by default.

In [4]:
print("| state 1 > < state 2 | = " , state_1 * state_2)

| state 1 > < state 2 | =  HilbertOperator
Re=
[[ 0.13766273  0.08920391 -0.06410887  0.03841083]
 [-0.09717941 -0.07032561  0.03920967 -0.00989339]
 [-0.24990439 -0.01456018  0.23754086 -0.41483372]
 [-0.21393467 -0.08768083  0.14151299 -0.17899253]]
Im:
[[-0.01857179 -0.10003659 -0.06370059  0.20089113]
 [ 0.00139627  0.0620355   0.04960722 -0.14275895]
 [ 0.26845007  0.35358859  0.02266833 -0.34574636]
 [ 0.11000793  0.21491686  0.06685491 -0.30564766]]


Ah look, taking the outer product has produced a new class: a `HilbertOperator`. We can combine multiplications between states and operators, but note that Python will associate them to the left, not to the right as we often like to, so you may need to put in some parentheses.



In [5]:
outer_product = state_1 * state_2
print("< state 1 | ( | state 1 > < state 2 | ) | state 2> = " , state_1.dagger() * outer_product * state_2.dagger())

< state 1 | ( | state 1 > < state 2 | ) | state 2> =  [[1.-4.16333634e-17j]]


We can make our own operator from scratch too. Operators are always instantiated to the identity. If you want to instantiate to zero, use the `.zero()` method. We can apply the Pauli spin operators to individual particles, and the isospin operators (if the isospin DOFs are included).

In [11]:
operator = HilbertOperator(n_particles=2, isospin=False).apply_sigma(particle_index=0, dimension=0)
print(operator)


HilbertOperator
Re=
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
Im:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In the tensor product space formalism, this is the one-body operator $\hat{\sigma}_{x} \otimes I := \hat{\sigma}_{1x}$. Note that indices start at zero: particle $i$ is indexed $i-1$.

In the full Hilbert space, states and operators can be added together to produce states and operators respectively. This is not the case with `ProductState` and `ProductOperator`, which I will get to later.

Let's construct the two-body spin interaction operator for our two particles: $\vec{\sigma}_1 \cdot \vec{\sigma}_2$.

In [15]:
si_dot_sj = HilbertOperator(n_particles=2,isospin=False).zero()
for dimension in (0,1,2):
    si_dot_sj += HilbertOperator(n_particles=2,isospin=False).apply_sigma(0,dimension).apply_sigma(1,dimension)

print(si_dot_sj)

HilbertOperator
Re=
[[ 1.  0.  0.  0.]
 [ 0. -1.  2.  0.]
 [ 0.  2. -1.  0.]
 [ 0.  0.  0.  1.]]
Im:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
